In [ ]:
import pandas as pd
import csv
import re

In [ ]:
records_dataframe = pd.read_pickle('working_files/records_dataframe.pkl')

In [ ]:
records_dataframe.head()

This code takes the Pre-Crash data string and converts it to table format.

In [ ]:
data = records_dataframe['Pre-Crash Data'].iloc[1]

# create a list of rows from the data
rows = data.split('\n')

event_1_pre_crash_data = []
for i in range(1, len(rows)):
    # use strip() to remove any extra white space or line breaks from the row
    row = rows[i].strip()
    if not row:
        # skip empty rows
        continue
    event_1_pre_crash_data.append(next(csv.reader([row])))

analysis_table_dataframe = pd.DataFrame(event_1_pre_crash_data, columns=event_1_pre_crash_data[0])
analysis_table_dataframe = analysis_table_dataframe.iloc[1:].reset_index(drop=True)
analysis_table_dataframe

In [ ]:
# reset the DataFrame index to avoid errors during iteration and remove empty rows that don't contain records
records_dataframe.reset_index(drop=True, inplace=True)

for i in range(len(records_dataframe)):
    # get the Pre-Crash Data for the current row
    data = records_dataframe['Pre-Crash Data'].iloc[i]

    # check if the data is a string to prevent string errors
    if not isinstance(data, str):
        # if not, set the value to NaN and skip to the next iteration
        continue

    # create a list of rows from the data
    rows = data.split('\n')

    event_1_pre_crash_data = []
    for j in range(1, len(rows)):
        try:
            # use a try/except block to catch any errors
            event_1_pre_crash_data.append(next(csv.reader([rows[j]])))
        except csv.Error:
            # if an error occurs, skip to the next iteration
            continue

    analysis_table_dataframe['Pre-Crash Dataframe'] = pd.DataFrame(event_1_pre_crash_data, columns=event_1_pre_crash_data[0])

    # save the DataFrame to a new variable
    # vin = records_dataframe['VIN'].iloc[i]
    # pd.to_pickle(analysis_table_dataframe, f'{vin}_first_record_pre_crash_data.pkl')

In [ ]:
# reset the DataFrame index to avoid errors during iteration and remove empty rows that don't contain records
records_dataframe.reset_index(drop=True, inplace=True)

# initialize an empty dataframe for each unique VIN
unique_vins = records_dataframe['VIN'].unique()
pre_crash_dataframes = {vin: pd.DataFrame() for vin in unique_vins}

for i in range(len(records_dataframe)):
    # get the Pre-Crash Data for the current row
    data = records_dataframe['Pre-Crash Data'].iloc[i]

    # check if the data is a string to prevent string errors
    if not isinstance(data, str):
        # if not, set the value to NaN and skip to the next iteration
        continue

    # create a list of rows from the data
    rows = data.split('\n')

    event_1_pre_crash_data = []
    for j in range(1, len(rows)):
        try:
            # use a try/except block to catch any errors
            event_1_pre_crash_data.append(next(csv.reader([rows[j]])))
        except csv.Error:
            # if an error occurs, skip to the next iteration
            continue

    # save the DataFrame to the pre_crash_dataframes dictionary with the corresponding VIN as the key
    vin = records_dataframe['VIN'].iloc[i]
    pre_crash_dataframes[vin] = pd.DataFrame(event_1_pre_crash_data[1:], columns=event_1_pre_crash_data[0]).dropna()

# add the DataFrame dictionary to the records_dataframe
records_dataframe['Pre-Crash Dataframes'] = [pre_crash_dataframes[vin] if vin in pre_crash_dataframes else pd.DataFrame() for vin in records_dataframe['VIN']]

In [ ]:
records_dataframe['Pre-Crash Dataframes'][1]

In [ ]:
pd.to_pickle(records_dataframe, 'working_files/records_dataframe.pkl')